How to set up selenium and client?

In [1]:
var importer = require('../Core');

var client, results, getCredentials, loginGoogle, getAllXPath;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client',
        'decrypt password',
        'log in Google',
        'get all elements'
    ])
    .then(r => {
        results = r;
        return r[0].runInNewContext();
    })
    .then(r => {
        client = r();
        getCredentials = results[1].runInNewContext({client});
        loginGoogle = results[2].runInNewContext({client, getCredentials});
        getAllXPath = results[3].runInNewContext({client});
        return client;
    });
};
module.exports = getClient;
getClient;



[Function: getClient]

In [2]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost


How to download google timeline data using webdriver?

In [3]:
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
    .url('https://www.google.com/maps/timeline')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
               .then(url => url.indexOf('timeline') > -1, 20000, ''))
    .catch(function (e) {
        console.log(e);
    })
    .pause(3000)
    .click('button[jsaction="header.select-today"]')
    .pause(2000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


[Function: getGoogleTimeline]

In [4]:
var readAllPages = () => {
    // TODO: add stop conditions
    var oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(
        Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    dayKey = '';
    var promises = [];
    for (var c = 0; c < days; c++) {
        var result;
        promises[promises.length] = (resolve) => readTimelinePage()
            .then(r => result = r)
            .click('.previous-date-range-button')
            .pause(2000)
            .then(() => {
                if(typeof(result) == 'undefined') {
                    resolve();
                }
                console.log(result);
                fs.writeFileSync('/Users/briancullinan/Timeline/timeline-'
                         + result[0].timeline + '_.json', JSON.stringify(result, null, 4))
                resolve(result);
            })
    }
    return promises;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


[Function: readAllPages]

In [5]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var readTimelinePage = () => {
    return client.getAllXPath({
        day: '//*[contains(@class,"timeline-subtitle")]//text()',
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
    .then(r => {
        console.log(r.day);
        if(typeof r.day != 'string') {
            return;
        }
        var currDate = chrono.parseDate(r.day);
        var newKey = currDate.getDate() 
            + months[currDate.getMonth()] 
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        
        return r.items.map(i => {
            var timelineData = (i.data + '').split(':');
            var start, end, length;
            if(timelineData[0]) {
                start = new Date(parseFloat(timelineData[1]));
                end = new Date(parseFloat(timelineData[2]));
                length = end.getTime() - start.getTime();
            } else {
                start = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[0]);
                end = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[1]);
                length = 0;
            }
            return ({
                type: 'timeline',
                timeline: dayKey,
                name: i.title,
                location: i.location,
                time: start,
                length: isNaN(length) ? 0 : length
            })
        })
    })
    .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


[Function: readTimelinePage]

In [ ]:
var fs = require('fs');
var importer = require('../Core');

$$.async()
client.getGoogleTimeline()
    .then((r) => importer.runAllPromises(readAllPages()))
    .catch(e => $$.sendError(e));



Google: Logging timeline history
[]
undefined
Thursday, September 21, 2017
[ { type: 'timeline',
    timeline: '21Sep17',
    name: [],
    location: [],
    time: 2017-09-21T19:00:00.000Z,
    length: 0 },
  { type: 'timeline',
    timeline: '21Sep17',
    name: '6934 E Sandra Terrace',
    location: 'Scottsdale, AZ 85254',
    time: 2017-09-21T14:18:00.000Z,
    length: 0 },
  { type: 'timeline',
    timeline: '21Sep17',
    name: [ ' ', 'Driving', ' - 5.6 mi' ],
    location: [],
    time: 2017-09-21T14:18:36.407Z,
    length: 664593 },
  { type: 'timeline',
    timeline: '21Sep17',
    name: 'Gainey Ranch Corporate Center',
    location: 'Swiftpage',
    time: 2017-09-21T14:29:00.000Z,
    length: 0 },
  { type: 'timeline',
    timeline: '21Sep17',
    name: [],
    location: [],
    time: 2017-09-21T19:00:00.000Z,
    length: 0 },
  { type: 'timeline',
    timeline: '21Sep17',
    name: [ ' ', 'Driving', ' - 3.5 mi' ],
    location: [],
    time: 2017-09-21T19:21:36.928Z,
    leng

In [4]:
$$.async();
client.windowHandles()
.then(h => $$.sendResult(h))
.catch(e => $$.sendError(e));

TypeError: Cannot read property 'windowHandles' of undefined

In [ ]:
var locationCache = {};
var dayKey; var count = 0;
$$.async();
importer.streamJson('/Users/briancullinan/Downloads/Takeout 5/Location History/Location History.json', [true, {emitPath: true}], (match) => {
    if(match.path[0] === 'locations') {
        var currDate = new Date(parseInt(match.value.timestampMs));
        var newKey = currDate.getDate() + months[currDate.getMonth()]
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        var newRow = Object.assign({
                time: currDate,
                type: 'location',
                location: dayKey,
        }, match.value);
        var cache = locationCache[newRow.location];
        if(typeof cache === 'undefined') {
            cache = locationCache[newRow.location] = [];
        }
        cache[cache.length] = newRow;
    }
})
    .then(() => Object.keys(locationCache).forEach(r => fs.writeFileSync(
        '/Users/briancullinan/Timeline/location-' 
        + locationCache[r][0].location + '.json',
        JSON.stringify(locationCache[r], null, 4))))

    .then(() => $$.sendResult(locationCache[dayKey]))
    .catch(e => $$.sendError(e));

